<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#KEYS" data-toc-modified-id="KEYS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>KEYS</a></span></li><li><span><a href="#Prepping-read-count-data" data-toc-modified-id="Prepping-read-count-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepping read count data</a></span><ul class="toc-item"><li><span><a href="#Combining-unnormalized-raw-featurecounts" data-toc-modified-id="Combining-unnormalized-raw-featurecounts-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Combining unnormalized raw featurecounts</a></span></li><li><span><a href="#Gathering-fragment-lengths-from-Featurecounts" data-toc-modified-id="Gathering-fragment-lengths-from-Featurecounts-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Gathering fragment lengths from Featurecounts</a></span></li></ul></li><li><span><a href="#Installing-packages-and-loading-libraries" data-toc-modified-id="Installing-packages-and-loading-libraries-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Installing packages and loading libraries</a></span></li><li><span><a href="#Size-Factors" data-toc-modified-id="Size-Factors-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Size Factors</a></span><ul class="toc-item"><li><span><a href="#DESeq2-normalizing-raw-data" data-toc-modified-id="DESeq2-normalizing-raw-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>DESeq2 normalizing raw data</a></span></li><li><span><a href="#Adding-GC-content-to-DESeq2-normalized-gene-counts" data-toc-modified-id="Adding-GC-content-to-DESeq2-normalized-gene-counts-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Adding GC content to DESeq2 normalized gene counts</a></span></li></ul></li><li><span><a href="#DESeq2---Differential-Expression-Analysis" data-toc-modified-id="DESeq2---Differential-Expression-Analysis-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>DESeq2 - Differential Expression Analysis</a></span><ul class="toc-item"><li><span><a href="#CC-2931" data-toc-modified-id="CC-2931-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>CC-2931</a></span></li><li><span><a href="#CC-2344" data-toc-modified-id="CC-2344-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>CC-2344</a></span></li></ul></li></ul></div>

## KEYS

In [1]:
import json
import pandas as pd
import numpy as np
from numpy import random
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator
from matplotlib.gridspec import GridSpec
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import mannwhitneyu
from scipy.stats import fligner
from scipy.stats import skewtest
stats.junk = lambda chisq, df: stats.chi2.sf(chisq, df)
import csv
import gffpandas.gffpandas as gffpd
import sklearn
from sklearn.metrics import r2_score
import os
import statistics
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
%load_ext rpy2.ipython

CC2344 = ["CC2344-L1", "CC2344-L2", "CC2344-L3", "CC2344-L4", "CC2344-L5", "CC2344-L6", "CC2344-L7", "CC2344-L8", "CC2344-L9", "CC2344-L10", "CC2344-L11", "CC2344-L12", "CC2344-L13", "CC2344-L14", "CC2344-L15"]
CC2931 = ["CC2931-L1", "CC2931-L2", "CC2931-L3", "CC2931-L4", "CC2931-L5", "CC2931-L6", "CC2931-L7", "CC2931-L9", "CC2931-L10", "CC2931-L11", "CC2931-L13", "CC2931-L14", "CC2931-L15"]

#### Dataframe recording the generation time per sample ####
dic_gen = {}
generation = pd.read_csv('/research/projects/chlamydomonas/MAexpression/genome_info/mutation_info/Mutation_Fitness.txt', delimiter = '\t')
generation['Sample'] = generation['Sample'].str.replace('_', '-L')
generation = generation.loc[generation['Sample'].isin(CC2344 + CC2931)]

for i in generation.index.values:
    dic_gen[generation.at[i,'Sample']] = generation.at[i, 'Generation']
generations = pd.Series(dic_gen)

#### Dataframe recording the number of mutations per sample ####
all_mutations = pd.read_csv('/research/projects/chlamydomonas/MAexpression/genome_info/mutation_info/all_mutations.csv', delimiter = '\t')
dic_mut = {maline:all_mutations.loc[all_mutations['sample'] == maline].count()[0] for maline in all_mutations['sample'].values.tolist()}
mutations = pd.Series(dic_mut)

## Prepping read count data

### Combining unnormalized raw featurecounts

In [2]:
#################
#### CC-2344 ####
#################

#### NAMES OF GENE COUNT FILES ####
keyword = 'genes_count'
label = []
for fname in os.listdir('/research/projects/chlamydomonas/MAexpression/data/gene_count/'):
    if keyword in fname:
        fname = fname.replace('_genes_count','')
        if 'CC2344' in fname:
            label.append(fname)
            
#### CC2344 - UNNORMALIZED GENE COUNTS ####
gene_count = pd.DataFrame()
for i in label:
    section = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/gene_count/' + i + '_genes_count', delimiter = '\t', skiprows = 1)
    section.columns = ['geneid', 'chr', 'start', 'end', 'strand', 'length', i]
    section = section[['geneid', i]]
    gene_count = pd.concat([gene_count, section], axis = 1)
gene_count = gene_count.loc[:,~gene_count.columns.duplicated()]
gene_count = gene_count.set_index('geneid')
gene_count.columns = pd.Series(gene_count.columns).str.split('.', expand = True)[1]

#### EXPORTING ####
gene_count.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/unnormalized_CC2344_raw.txt', sep = '\t', index = True, header = True)

#################
#### CC-2931 ####
#################

#### NAMES OF GENE COUNT FILES ####
keyword = 'genes_count'
label = []
for fname in os.listdir('/research/projects/chlamydomonas/MAexpression/data/gene_count/'):
    if keyword in fname:
        fname = fname.replace('_genes_count','')
        if 'CC2931' in fname:
            label.append(fname)
            
#### CC2931 - UNNORMALIZED GENE COUNTS ####
gene_count = pd.DataFrame()
for i in label:
    section = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/gene_count/' + i + '_genes_count', delimiter = '\t', skiprows = 1)
    section.columns = ['geneid', 'chr', 'start', 'end', 'strand', 'length', i]
    section = section[['geneid', i]]
    gene_count = pd.concat([gene_count, section], axis = 1)
gene_count = gene_count.loc[:,~gene_count.columns.duplicated()]
gene_count = gene_count.set_index('geneid')
gene_count.columns = pd.Series(gene_count.columns).str.split('.', expand = True)[1]

#### EXPORTING ####
gene_count.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/unnormalized_CC2931_raw.txt', sep = '\t', index = True, header = True)

### Gathering fragment lengths from Featurecounts

In [7]:
fragment_length = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/gene_count/198.CC2344-L15-rep3_genes_count', delimiter = '\t', skiprows = 1, usecols = ['Geneid','Length'])
fragment_length = fragment_length.set_index('Geneid')

# EXPORT DATA
fragment_length.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/fpkm/fragment_length.csv', sep = '\t', index = True, index_label= 'index', header = True)

## Installing packages and loading libraries

In [3]:
%%R
## TIDYVERSE
library(tidyverse)

## DESEQ2
library("BiocParallel")
register(MulticoreParam(4))
library("DESeq2")

R[write to console]: ── Attaching packages ───────────────────────────────── tidyverse 1.3.1.9000 ──

R[write to console]: ✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.2.0     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.0.0     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ───────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write

## Size Factors

In [11]:
%%R

################
#### CC2931 ####
################
CC2931_raw_counts <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/unnormalized_CC2931_raw.txt', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = "geneid")
counts <- CC2931_raw_counts[c(1:42)]

#### Assigning factors and conditions ####
condition <- c('ANC', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L9', 'L10', 'L11', 'L13', 'L14', 'L15')
factor <- colnames(counts)
columns <- factor
for (i in condition) {factor[grep((i), factor)] <- i}
factor[grep('L1.rep', factor)] <- 'L1'
coldata <- data.frame(factor, columns, row.names = 'columns')

#### Creating data object ####
CC2931_dds <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)
directory <- "/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/"
CC2931_dds <- estimateSizeFactors(CC2931_dds)
sizeFactors(CC2931_dds)

CC2931.ANC.rep1 CC2931.ANC.rep2 CC2931.ANC.rep3  CC2931.L1.rep1  CC2931.L1.rep2 
      1.8502940       1.0322654       0.8007557       2.3773915       1.3148481 
 CC2931.L1.rep3  CC2931.L2.rep1  CC2931.L2.rep2  CC2931.L2.rep3 CC2931.L15_rep1 
      1.4059812       0.5266999       0.9162189       1.0550166       1.4258836 
CC2931.L15_rep2 CC2931.L15_rep3  CC2931.L3_rep1  CC2931.L3_rep2  CC2931.L3_rep3 
      1.0288307       1.2657351       0.6031843       1.1181484       0.8886967 
 CC2931.L4_rep1  CC2931.L4_rep2  CC2931.L5_rep1  CC2931.L5_rep3  CC2931.L6_rep2 
      1.5233362       1.4395133       1.0948225       1.5069215       1.6812947 
 CC2931.L7_rep1  CC2931.L7_rep3  CC2931.L9_rep1 CC2931.L13_rep1  CC2931.L4_rep3 
      0.9991658       0.4242726       0.3583392       0.4939641       0.9496580 
 CC2931.L5_rep2  CC2931.L6_rep1  CC2931.L6_rep3  CC2931.L7_rep2 CC2931.L11_rep3 
      1.4963887       1.2747691       1.8245148       1.2956589       0.7424569 
 CC2931.L9_rep2  CC2931.L9_r

In [14]:
string = """CC2931.ANC.rep1 CC2931.ANC.rep2 CC2931.ANC.rep3  CC2931.L1.rep1  CC2931.L1.rep2 
      1.8502940       1.0322654       0.8007557       2.3773915       1.3148481 
 CC2931.L1.rep3  CC2931.L2.rep1  CC2931.L2.rep2  CC2931.L2.rep3 CC2931.L15_rep1 
      1.4059812       0.5266999       0.9162189       1.0550166       1.4258836 
CC2931.L15_rep2 CC2931.L15_rep3  CC2931.L3_rep1  CC2931.L3_rep2  CC2931.L3_rep3 
      1.0288307       1.2657351       0.6031843       1.1181484       0.8886967 
 CC2931.L4_rep1  CC2931.L4_rep2  CC2931.L5_rep1  CC2931.L5_rep3  CC2931.L6_rep2 
      1.5233362       1.4395133       1.0948225       1.5069215       1.6812947 
 CC2931.L7_rep1  CC2931.L7_rep3  CC2931.L9_rep1 CC2931.L13_rep1  CC2931.L4_rep3 
      0.9991658       0.4242726       0.3583392       0.4939641       0.9496580 
 CC2931.L5_rep2  CC2931.L6_rep1  CC2931.L6_rep3  CC2931.L7_rep2 CC2931.L11_rep3 
      1.4963887       1.2747691       1.8245148       1.2956589       0.7424569 
 CC2931.L9_rep2  CC2931.L9_rep3 CC2931.L13_rep2 CC2931.L10_rep1 CC2931.L13_rep3 
      0.3790061       1.0412170       1.1207979       0.5070304       1.2204579 
CC2931.L10_rep2 CC2931.L14_rep1 CC2931.L10_rep3 CC2931.L14_rep2 CC2931.L11_rep1 
      0.6088216       1.6119599       0.5004204       0.9491339       1.7082821 
CC2931.L14_rep3 CC2931.L11_rep2 
      0.5156808       1.9069923"""
names=[]
sfs = []
for i in string.strip().split():
    if i[0] =="C":names.append(i)
    else: sfs.append(float(i))

for i, j in zip(names, sfs):
    print(i, j)

CC2931.ANC.rep1 1.850294
CC2931.ANC.rep2 1.0322654
CC2931.ANC.rep3 0.8007557
CC2931.L1.rep1 2.3773915
CC2931.L1.rep2 1.3148481
CC2931.L1.rep3 1.4059812
CC2931.L2.rep1 0.5266999
CC2931.L2.rep2 0.9162189
CC2931.L2.rep3 1.0550166
CC2931.L15_rep1 1.4258836
CC2931.L15_rep2 1.0288307
CC2931.L15_rep3 1.2657351
CC2931.L3_rep1 0.6031843
CC2931.L3_rep2 1.1181484
CC2931.L3_rep3 0.8886967
CC2931.L4_rep1 1.5233362
CC2931.L4_rep2 1.4395133
CC2931.L5_rep1 1.0948225
CC2931.L5_rep3 1.5069215
CC2931.L6_rep2 1.6812947
CC2931.L7_rep1 0.9991658
CC2931.L7_rep3 0.4242726
CC2931.L9_rep1 0.3583392
CC2931.L13_rep1 0.4939641
CC2931.L4_rep3 0.949658
CC2931.L5_rep2 1.4963887
CC2931.L6_rep1 1.2747691
CC2931.L6_rep3 1.8245148
CC2931.L7_rep2 1.2956589
CC2931.L11_rep3 0.7424569
CC2931.L9_rep2 0.3790061
CC2931.L9_rep3 1.041217
CC2931.L13_rep2 1.1207979
CC2931.L10_rep1 0.5070304
CC2931.L13_rep3 1.2204579
CC2931.L10_rep2 0.6088216
CC2931.L14_rep1 1.6119599
CC2931.L10_rep3 0.5004204
CC2931.L14_rep2 0.9491339
CC2931.L11_re

In [12]:
%%R
################
#### CC2344 ####
################
CC2344_raw_counts <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/unnormalized_CC2344_raw.txt', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = 'geneid')
counts <- CC2344_raw_counts[c(1:48)]

#### Assigning factors and conditions ####
condition <- c('ANC', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12', 'L13', 'L14', 'L15')
factor <- colnames(counts)
columns <- factor
for (i in condition) {factor[grep((i), factor)] <- i}
factor[grep('L1.rep', factor)] <- 'L1'
coldata <- data.frame(factor, columns, row.names = 'columns')

#### Creating data object ####
CC2344_dds <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)
directory <- "/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/"
CC2344_dds <- estimateSizeFactors(CC2344_dds)
sizeFactors(CC2344_dds)

CC2344.L15.rep3 CC2344.L15.rep2 CC2344.ANC.rep2  CC2344.L1.rep1  CC2344.L1.rep3 
      1.1737230       1.3257651       1.3987882       1.8477988       1.9994374 
 CC2344.L2.rep2  CC2344.L3.rep1  CC2344.L3.rep3  CC2344.L4.rep2  CC2344.L5.rep1 
      0.4575387       0.7316950       0.6404584       1.4682027       0.2274608 
 CC2344.L5.rep3  CC2344.L6.rep2  CC2344.L7.rep1  CC2344.L8.rep1  CC2344.L8.rep3 
      0.7020427       0.4681475       0.5782607       0.9354567       0.7032777 
 CC2344.L9.rep2 CC2344.L10.rep1 CC2344.L10.rep3 CC2344.L11.rep2 CC2344.L12.rep1 
      3.2269203       0.8035964       0.3408698       1.2464077       1.2621707 
CC2344.L12.rep3 CC2344.L13.rep2 CC2344.L14.rep1 CC2344.L14.rep3  CC2344.L7.rep3 
      0.5445667       1.4719460       1.4588763       1.1712411       0.5062961 
CC2344.ANC.rep1 CC2344.ANC.rep3  CC2344.L1.rep2  CC2344.L2.rep1  CC2344.L2.rep3 
      0.5721730       1.9936471       0.9835393       0.7979124       1.3583711 
 CC2344.L3.rep2  CC2344.L4.r

In [13]:
string = """CC2344.L15.rep3 CC2344.L15.rep2 CC2344.ANC.rep2  CC2344.L1.rep1  CC2344.L1.rep3 
      1.1737230       1.3257651       1.3987882       1.8477988       1.9994374 
 CC2344.L2.rep2  CC2344.L3.rep1  CC2344.L3.rep3  CC2344.L4.rep2  CC2344.L5.rep1 
      0.4575387       0.7316950       0.6404584       1.4682027       0.2274608 
 CC2344.L5.rep3  CC2344.L6.rep2  CC2344.L7.rep1  CC2344.L8.rep1  CC2344.L8.rep3 
      0.7020427       0.4681475       0.5782607       0.9354567       0.7032777 
 CC2344.L9.rep2 CC2344.L10.rep1 CC2344.L10.rep3 CC2344.L11.rep2 CC2344.L12.rep1 
      3.2269203       0.8035964       0.3408698       1.2464077       1.2621707 
CC2344.L12.rep3 CC2344.L13.rep2 CC2344.L14.rep1 CC2344.L14.rep3  CC2344.L7.rep3 
      0.5445667       1.4719460       1.4588763       1.1712411       0.5062961 
CC2344.ANC.rep1 CC2344.ANC.rep3  CC2344.L1.rep2  CC2344.L2.rep1  CC2344.L2.rep3 
      0.5721730       1.9936471       0.9835393       0.7979124       1.3583711 
 CC2344.L3.rep2  CC2344.L4.rep1  CC2344.L4.rep3  CC2344.L5.rep2  CC2344.L6.rep1 
      1.8257090       0.9201224       0.7725692       1.2138964       0.7650153 
 CC2344.L6.rep3  CC2344.L7.rep2  CC2344.L8.rep2  CC2344.L9.rep1  CC2344.L9.rep3 
      1.3146830       0.6315503       1.6609855       2.0764098       1.9049752 
CC2344.L10.rep2 CC2344.L11.rep1 CC2344.L11.rep3 CC2344.L12.rep2 CC2344.L13.rep1 
      0.9330056       1.6455449       1.2592621       1.1726861       0.8356573 
CC2344.L13.rep3 CC2344.L14.rep2 CC2344.L15.rep1 
      1.2341825       1.2795932       0.7815065"""
names=[]
sfs = []
for i in string.strip().split():
    if i[0] =="C":names.append(i)
    else: sfs.append(float(i))

for i, j in zip(names, sfs):
    print(i, j)

CC2344.L15.rep3 1.173723
CC2344.L15.rep2 1.3257651
CC2344.ANC.rep2 1.3987882
CC2344.L1.rep1 1.8477988
CC2344.L1.rep3 1.9994374
CC2344.L2.rep2 0.4575387
CC2344.L3.rep1 0.731695
CC2344.L3.rep3 0.6404584
CC2344.L4.rep2 1.4682027
CC2344.L5.rep1 0.2274608
CC2344.L5.rep3 0.7020427
CC2344.L6.rep2 0.4681475
CC2344.L7.rep1 0.5782607
CC2344.L8.rep1 0.9354567
CC2344.L8.rep3 0.7032777
CC2344.L9.rep2 3.2269203
CC2344.L10.rep1 0.8035964
CC2344.L10.rep3 0.3408698
CC2344.L11.rep2 1.2464077
CC2344.L12.rep1 1.2621707
CC2344.L12.rep3 0.5445667
CC2344.L13.rep2 1.471946
CC2344.L14.rep1 1.4588763
CC2344.L14.rep3 1.1712411
CC2344.L7.rep3 0.5062961
CC2344.ANC.rep1 0.572173
CC2344.ANC.rep3 1.9936471
CC2344.L1.rep2 0.9835393
CC2344.L2.rep1 0.7979124
CC2344.L2.rep3 1.3583711
CC2344.L3.rep2 1.825709
CC2344.L4.rep1 0.9201224
CC2344.L4.rep3 0.7725692
CC2344.L5.rep2 1.2138964
CC2344.L6.rep1 0.7650153
CC2344.L6.rep3 1.314683
CC2344.L7.rep2 0.6315503
CC2344.L8.rep2 1.6609855
CC2344.L9.rep1 2.0764098
CC2344.L9.rep3 1.9

### DESeq2 normalizing raw data

In [4]:
%%R

################
#### CC2931 ####
################
CC2931_raw_counts <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/unnormalized_CC2931_raw.txt', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = "geneid")
counts <- CC2931_raw_counts[c(1:42)]

#### Assigning factors and conditions ####
condition <- c('ANC', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L9', 'L10', 'L11', 'L13', 'L14', 'L15')
factor <- colnames(counts)
columns <- factor
for (i in condition) {factor[grep((i), factor)] <- i}
factor[grep('L1.rep', factor)] <- 'L1'
coldata <- data.frame(factor, columns, row.names = 'columns')

#### Creating data object ####
CC2931_dds <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)
directory <- "/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/"
CC2931_dds <- estimateSizeFactors(CC2931_dds)
counts <- counts(CC2931_dds, normalized=TRUE)
write.table(counts, paste(directory, "CC2931_deseq_normalized", ".txt", sep=""), sep="\t", quote = F)

################
#### CC2344 ####
################
CC2344_raw_counts <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/unnormalized_CC2344_raw.txt', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = 'geneid')
counts <- CC2344_raw_counts[c(1:48)]

#### Assigning factors and conditions ####
condition <- c('ANC', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12', 'L13', 'L14', 'L15')
factor <- colnames(counts)
columns <- factor
for (i in condition) {factor[grep((i), factor)] <- i}
factor[grep('L1.rep', factor)] <- 'L1'
coldata <- data.frame(factor, columns, row.names = 'columns')

#### Creating data object ####
CC2344_dds <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)
directory <- "/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/"
CC2344_dds <- estimateSizeFactors(CC2344_dds)
counts <- counts(CC2344_dds, normalized=TRUE)
write.table(counts, paste(directory, "CC2344_deseq_normalized", ".txt", sep=""), sep="\t", quote = F)

### Adding GC content to DESeq2 normalized gene counts

In [5]:
## Opening GC length and fragment length
gc_content = pd.read_csv('/research/projects/chlamydomonas/MAexpression/genome_info/v6_genome_plus_anno/GC_lengths.tsv', delimiter = '\t')
fragment_length = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/fpkm/fragment_length.csv', delimiter = '\t')
fragment_length = fragment_length.set_index('index')

## Opening DESeq2 normalized gene counts
CC2931_normalized_GC = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2931_deseq_normalized.txt', delimiter = '\t')
CC2344_normalized_GC = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2344_deseq_normalized.txt', delimiter = '\t')

CC2931_normalized_GC = CC2931_normalized_GC.join([fragment_length, gc_content['GC']])
CC2344_normalized_GC = CC2344_normalized_GC.join([fragment_length, gc_content['GC']])

## Export ##
CC2931_normalized_GC.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2931_deseq_normalized_wt_GC', sep = '\t', index = True, header = True)
CC2344_normalized_GC.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/CC2344_deseq_normalized_wt_GC', sep = '\t', index = True, header = True)

## DESeq2 - Differential Expression Analysis

### CC-2931

Creating DESeq2 data object

In [6]:
%%R

CC2931_raw_counts <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/unnormalized_CC2931_raw.txt', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = "geneid")
counts <- CC2931_raw_counts[c(1:42)]

#### Assigning factors and conditions ####
condition <- c('ANC', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L9', 'L10', 'L11', 'L13', 'L14', 'L15')
factor <- colnames(counts)
columns <- factor
for (i in condition) {factor[grep((i), factor)] <- i}
factor[grep('L1.rep', factor)] <- 'L1'
coldata <- data.frame(factor, columns, row.names = 'columns')

#### Creating data object ####
CC2931_dds1 <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)

Running DESeq2

In [7]:
%%R

#### USING DESEQ2 #####
dds <- DESeq(CC2931_dds1, parallel = TRUE)
res_L1 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L1'), alpha = 0.05)
res_L2 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L2'), alpha = 0.05)
res_L3 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L3'), alpha = 0.05)
res_L4 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L4'), alpha = 0.05)
res_L5 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L5'), alpha = 0.05)
res_L6 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L6'), alpha = 0.05)
res_L7 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L7'), alpha = 0.05)
res_L9 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L9'), alpha = 0.05)
res_L10 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L10'), alpha = 0.05)
res_L11 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L11'), alpha = 0.05)
res_L13 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L13'), alpha = 0.05)
res_L14 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L14'), alpha = 0.05)
res_L15 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L15'), alpha = 0.05)


#### FILES OF DIFFERNTIALLY EXPRESSED GENES ####
resSig_L1 = res_L1[which(res_L1$padj < 0.05),]
resSig_L2 = res_L2[which(res_L2$padj < 0.05),]
resSig_L3 = res_L3[which(res_L3$padj < 0.05),]
resSig_L4 = res_L4[which(res_L4$padj < 0.05),]
resSig_L5 = res_L5[which(res_L5$padj < 0.05),]
resSig_L6 = res_L6[which(res_L6$padj < 0.05),]
resSig_L7 = res_L7[which(res_L7$padj < 0.05),]
resSig_L9 = res_L9[which(res_L9$padj < 0.05),]
resSig_L10 = res_L10[which(res_L10$padj < 0.05),]
resSig_L11 = res_L11[which(res_L11$padj < 0.05),]
resSig_L13 = res_L13[which(res_L13$padj < 0.05),]
resSig_L14 = res_L14[which(res_L14$padj < 0.05),]
resSig_L15 = res_L15[which(res_L15$padj < 0.05),]

#### EXPORTING FILES OF DIFFERNTIALLY EXPRESSED GENES ####
write.csv(as.data.frame(resSig_L1), file="DEGs/degs/CC2931.resSig_L1.genes")
write.csv(as.data.frame(resSig_L2), file="DEGs/degs/CC2931.resSig_L2.genes")
write.csv(as.data.frame(resSig_L3), file="DEGs/degs/CC2931.resSig_L3.genes")
write.csv(as.data.frame(resSig_L4), file="DEGs/degs/CC2931.resSig_L4.genes")
write.csv(as.data.frame(resSig_L5), file="DEGs/degs/CC2931.resSig_L5.genes")
write.csv(as.data.frame(resSig_L6), file="DEGs/degs/CC2931.resSig_L6.genes")
write.csv(as.data.frame(resSig_L7), file="DEGs/degs/CC2931.resSig_L7.genes")
write.csv(as.data.frame(resSig_L9), file="DEGs/degs/CC2931.resSig_L9.genes")
write.csv(as.data.frame(resSig_L10), file="DEGs/degs/CC2931.resSig_L10.genes")
write.csv(as.data.frame(resSig_L11), file="DEGs/degs/CC2931.resSig_L11.genes")
write.csv(as.data.frame(resSig_L13), file="DEGs/degs/CC2931.resSig_L13.genes")
write.csv(as.data.frame(resSig_L14), file="DEGs/degs/CC2931.resSig_L14.genes")
write.csv(as.data.frame(resSig_L15), file="DEGs/degs/CC2931.resSig_L15.genes")

#### EXPORTING LOG2FOLD CHANGE RESULTS ####
write.table(as.data.frame(res_L1), 'genes_log2fold/CC2931-L1',  sep="\t", quote = F)
write.table(as.data.frame(res_L2), 'genes_log2fold/CC2931-L2',  sep="\t", quote = F)
write.table(as.data.frame(res_L3), 'genes_log2fold/CC2931-L3',  sep="\t", quote = F)
write.table(as.data.frame(res_L4), 'genes_log2fold/CC2931-L4',  sep="\t", quote = F)
write.table(as.data.frame(res_L5), 'genes_log2fold/CC2931-L5',  sep="\t", quote = F)
write.table(as.data.frame(res_L6), 'genes_log2fold/CC2931-L6',  sep="\t", quote = F)
write.table(as.data.frame(res_L7), 'genes_log2fold/CC2931-L7',  sep="\t", quote = F)
write.table(as.data.frame(res_L9), 'genes_log2fold/CC2931-L9',  sep="\t", quote = F)
write.table(as.data.frame(res_L10), 'genes_log2fold/CC2931-L10',  sep="\t", quote = F)
write.table(as.data.frame(res_L11), 'genes_log2fold/CC2931-L11',  sep="\t", quote = F)
write.table(as.data.frame(res_L13), 'genes_log2fold/CC2931-L13',  sep="\t", quote = F)
write.table(as.data.frame(res_L14), 'genes_log2fold/CC2931-L14',  sep="\t", quote = F)
write.table(as.data.frame(res_L15), 'genes_log2fold/CC2931-L15',  sep="\t", quote = F)

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates: 4 workers

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates, fitting model and testing: 4 workers



### CC-2344

Creating DESeq2 data object

In [8]:
%%R

CC2344_raw_counts <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/unnormalized_CC2344_raw.txt', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = 'geneid')
counts <- CC2344_raw_counts[c(1:48)]

#### Assigning factors and conditions ####
condition <- c('ANC', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12', 'L13', 'L14', 'L15')
factor <- colnames(counts)
columns <- factor
for (i in condition) {factor[grep((i), factor)] <- i}
factor[grep('L1.rep', factor)] <- 'L1'
coldata <- data.frame(factor, columns, row.names = 'columns')

#### Creating data object ####
CC2344_dds1 <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)

Running DESeq2

In [ ]:
%%R

#### USING DESEQ2 #####
dds <- DESeq(CC2344_dds1, parallel = TRUE)
res_L1 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L1'), alpha = 0.05)
res_L2 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L2'), alpha = 0.05)
res_L3 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L3'), alpha = 0.05)
res_L4 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L4'), alpha = 0.05)
res_L5 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L5'), alpha = 0.05)
res_L6 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L6'), alpha = 0.05)
res_L7 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L7'), alpha = 0.05)
res_L8 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L8'), alpha = 0.05)
res_L9 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L9'), alpha = 0.05)
res_L10 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L10'), alpha = 0.05)
res_L11 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L11'), alpha = 0.05)
res_L12 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L12'), alpha = 0.05)
res_L13 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L13'), alpha = 0.05)
res_L14 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L14'), alpha = 0.05)
res_L15 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'ANC','L15'), alpha = 0.05)

#### FILES OF DIFFERNTIALLY EXPRESSED GENES ####
resSig_L1 = res_L1[which(res_L1$padj < 0.05),]
resSig_L2 = res_L2[which(res_L2$padj < 0.05),]
resSig_L3 = res_L3[which(res_L3$padj < 0.05),]
resSig_L4 = res_L4[which(res_L4$padj < 0.05),]
resSig_L5 = res_L5[which(res_L5$padj < 0.05),]
resSig_L6 = res_L6[which(res_L6$padj < 0.05),]
resSig_L7 = res_L7[which(res_L7$padj < 0.05),]
resSig_L8 = res_L8[which(res_L8$padj < 0.05),]
resSig_L9 = res_L9[which(res_L9$padj < 0.05),]
resSig_L10 = res_L10[which(res_L10$padj < 0.05),]
resSig_L11 = res_L11[which(res_L11$padj < 0.05),]
resSig_L12 = res_L12[which(res_L12$padj < 0.05),]
resSig_L13 = res_L13[which(res_L13$padj < 0.05),]
resSig_L14 = res_L14[which(res_L14$padj < 0.05),]
resSig_L15 = res_L15[which(res_L15$padj < 0.05),]

#### EXPORTING FILES OF DIFFERNTIALLY EXPRESSED GENES ####
write.csv(as.data.frame(resSig_L1), file="DEGs/degs/CC2344.resSig_L1.genes")
write.csv(as.data.frame(resSig_L2), file="DEGs/degs/CC2344.resSig_L2.genes")
write.csv(as.data.frame(resSig_L3), file="DEGs/degs/CC2344.resSig_L3.genes")
write.csv(as.data.frame(resSig_L4), file="DEGs/degs/CC2344.resSig_L4.genes")
write.csv(as.data.frame(resSig_L5), file="DEGs/degs/CC2344.resSig_L5.genes")
write.csv(as.data.frame(resSig_L6), file="DEGs/degs/CC2344.resSig_L6.genes")
write.csv(as.data.frame(resSig_L7), file="DEGs/degs/CC2344.resSig_L7.genes")
write.csv(as.data.frame(resSig_L8), file="DEGs/degs/CC2344.resSig_L8.genes")
write.csv(as.data.frame(resSig_L9), file="DEGs/degs/CC2344.resSig_L9.genes")
write.csv(as.data.frame(resSig_L10), file="DEGs/degs/CC2344.resSig_L10.genes")
write.csv(as.data.frame(resSig_L11), file="DEGs/degs/CC2344.resSig_L11.genes")
write.csv(as.data.frame(resSig_L12), file="DEGs/degs/CC2344.resSig_L12.genes")
write.csv(as.data.frame(resSig_L13), file="DEGs/degs/CC2344.resSig_L13.genes")
write.csv(as.data.frame(resSig_L14), file="DEGs/degs/CC2344.resSig_L14.genes")
write.csv(as.data.frame(resSig_L15), file="DEGs/degs/CC2344.resSig_L15.genes")

#### EXPORTING LOG2FOLD CHANGE RESULTS ####
write.table(as.data.frame(res_L1), 'genes_log2fold/CC2344-L1',  sep="\t", quote = F)
write.table(as.data.frame(res_L2), 'genes_log2fold/CC2344-L2',  sep="\t", quote = F)
write.table(as.data.frame(res_L3), 'genes_log2fold/CC2344-L3',  sep="\t", quote = F)
write.table(as.data.frame(res_L4), 'genes_log2fold/CC2344-L4',  sep="\t", quote = F)
write.table(as.data.frame(res_L5), 'genes_log2fold/CC2344-L5',  sep="\t", quote = F)
write.table(as.data.frame(res_L6), 'genes_log2fold/CC2344-L6',  sep="\t", quote = F)
write.table(as.data.frame(res_L7), 'genes_log2fold/CC2344-L7',  sep="\t", quote = F)
write.table(as.data.frame(res_L8), 'genes_log2fold/CC2344-L8',  sep="\t", quote = F)
write.table(as.data.frame(res_L9), 'genes_log2fold/CC2344-L9',  sep="\t", quote = F)
write.table(as.data.frame(res_L10), 'genes_log2fold/CC2344-L10',  sep="\t", quote = F)
write.table(as.data.frame(res_L11), 'genes_log2fold/CC2344-L11',  sep="\t", quote = F)
write.table(as.data.frame(res_L12), 'genes_log2fold/CC2344-L12',  sep="\t", quote = F)
write.table(as.data.frame(res_L13), 'genes_log2fold/CC2344-L13',  sep="\t", quote = F)
write.table(as.data.frame(res_L14), 'genes_log2fold/CC2344-L14',  sep="\t", quote = F)
write.table(as.data.frame(res_L15), 'genes_log2fold/CC2344-L15',  sep="\t", quote = F)

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates: 4 workers

